# Probability

> Core probability utilities for RBE - normalization, sampling, entropy, and divergence measures

In [ ]:
#| default_exp rbe.probability

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from typing import Optional, Union, List
from fastcore.test import test_eq, test_close
from fastcore.all import *

## Basic Operations

Core probability operations following fast.ai style - short names, clear purpose.

In [ ]:
#| export
def normalize(probs):
    "Normalize `probs` to sum to 1"
    probs = np.asarray(probs)
    s = np.sum(probs)
    if s == 0: raise ValueError("Cannot normalize zero probabilities")
    return probs / s

def sample(probs, n=1, rng=None):
    "Sample `n` indices from `probs` distribution"
    if rng is None: rng = np.random.default_rng()
    probs = np.asarray(probs)
    if np.any(probs < 0): raise ValueError("Probabilities must be non-negative")
    probs = normalize(probs)
    return rng.choice(len(probs), size=n, p=probs)

In [ ]:
# Test basic operations
probs = [1, 2, 3]
normed = normalize(probs)
test_close(np.sum(normed), 1.0)
test_close(normed, [1/6, 2/6, 3/6])

# Test sampling
rng = np.random.default_rng(42)
samples = sample([0.1, 0.7, 0.2], n=1000, rng=rng)
assert len(samples) == 1000
assert np.all((samples >= 0) & (samples <= 2))

## Information Measures

Entropy and divergence measures for quantifying uncertainty and comparing distributions.

In [ ]:
#| export
def entropy(probs, base=2):
    "Calculate entropy of `probs` distribution in given `base`"
    probs = normalize(probs)
    probs = probs[probs > 0]  # Remove zeros to avoid log(0)
    if base == 2:
        return -np.sum(probs * np.log2(probs))
    elif base == 'e':
        return -np.sum(probs * np.log(probs))
    else:
        return -np.sum(probs * np.log(probs)) / np.log(base)

def kl_div(p, q, eps=1e-10):
    "KL divergence from `q` to `p`"
    p, q = normalize(p), normalize(q)
    # Add epsilon to avoid log(0)
    return np.sum(p * np.log((p + eps) / (q + eps)))

def js_div(p, q):
    "Jensen-Shannon divergence between `p` and `q`"
    p, q = normalize(p), normalize(q)
    m = 0.5 * (p + q)
    return 0.5 * kl_div(p, m) + 0.5 * kl_div(q, m)

In [ ]:
# Test entropy
uniform = [0.5, 0.5]
certain = [1.0, 0.0]
assert entropy(uniform) > entropy(certain)
test_close(entropy(uniform), 1.0)  # Maximum entropy for 2 outcomes

# Test KL divergence
p = [0.5, 0.5]
q = [0.5, 0.5]
test_close(kl_div(p, q), 0.0, eps=1e-10)  # Same distributions

# Test JS divergence (symmetric)
test_close(js_div(p, q), js_div(q, p))  # Should be symmetric

## Effective Sample Size

Measure of particle filter health - how many particles are effectively contributing.

In [ ]:
#| export
def eff_size(weights):
    "Calculate effective sample size of normalized `weights`"
    weights = normalize(weights)
    return 1.0 / np.sum(weights**2)

In [ ]:
# Test effective sample size
uniform_weights = np.ones(100) / 100
skewed_weights = np.zeros(100)
skewed_weights[0] = 1.0

test_close(eff_size(uniform_weights), 100.0)  # All particles contribute
test_close(eff_size(skewed_weights), 1.0)     # Only one particle

## Categorical Distribution Utilities

In [ ]:
#| export
def categorical(probs, labels=None):
    "Create categorical distribution from `probs` with optional `labels`"
    probs = normalize(probs)
    if labels is None:
        labels = list(range(len(probs)))
    return dict(zip(labels, probs))

def uniform(n):
    "Create uniform distribution over `n` outcomes"
    return np.ones(n) / n

def from_counts(counts):
    "Create probability distribution from `counts`"
    counts = np.asarray(counts)
    if np.any(counts < 0):
        raise ValueError("Counts must be non-negative")
    return normalize(counts)

In [ ]:
# Test categorical utilities
cat_dist = categorical([1, 2, 3], ['A', 'B', 'C'])
test_eq(cat_dist['A'], 1/6)
test_eq(cat_dist['B'], 2/6)
test_eq(cat_dist['C'], 3/6)

# Test uniform
u = uniform(4)
test_close(u, [0.25, 0.25, 0.25, 0.25])

# Test from_counts
probs = from_counts([10, 20, 30])
test_close(probs, [1/6, 2/6, 3/6])

## Export

In [ ]:
#| export
__all__ = [
    # Basic operations
    'normalize', 'sample',
    
    # Information measures
    'entropy', 'kl_div', 'js_div',
    
    # Effective sample size
    'eff_size',
    
    # Categorical utilities
    'categorical', 'uniform', 'from_counts'
]